In [26]:
from markitdown import MarkItDown
import pdfplumber
import os 
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter 
from pypdf import PdfReader, PdfWriter
import shlex
import requests
import json
import csv

Garder la dernière page du PDF : 

In [ ]:
def extract_last_page(input_path, output_path):
    # Read the original PDF
    reader = PdfReader(input_path)
    writer = PdfWriter()

    # Get the last page
    last_page = reader.pages[-1]
    writer.add_page(last_page)

    # Write it to a new PDF
    with open(output_path, "wb") as output_file:
        writer.write(output_file)

    print(f"Last page saved to: {output_path}")

In [18]:
def is_html(file_path):
    with open(file_path, "rb") as f:
        header = f.read(1024).lower()
        return b"<html" in header or b"<!doctype html" in header

Conversion PDF en Markdown

In [15]:
markitdown = MarkItDown()
result = markitdown.convert('pdfs\ACSA%201_2.pdf')
print(result.text_content)

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lisef\AppData\Local\Temp\ipykernel_16024\700113543.py:2: SyntaxWarning: invalid escape sequence '\A'
  result = markitdown.convert('pdfs\ACSA%201_2.pdf')


ACSA 1

CMA CGM weekly fixed-day service connecting Far East and West Coast South America

Busan

Yokohama

Busan

Yokohama

Shanghai
Ningbo

Shanghai
Ningbo

Keelung

Chiwan

Hong Kong

Manzanillo

Chancay
Callao

Arica

Iquique

INCA

Chancay
Callao

San Antonio
Lirquen

INCA2

Balboa

Posorja

Chancay
Callao
Callao

San Antonio

CMA CGM Strengths

•  Direct and unique product offering fastest transit time between Far East & West Coast South America (WCSA)
•  Direct call to San Antonio, offering the most competitive transit time on the market
•  Seamless connection with CMA CGM direct fixed-day weekly INCA services (to connect Arica/Iquique/Lirquen)
•  Designed for perishable and fresh fruit in reefer containers from West Coast South America to the Far East

ACSA 1

CMA CGM weekly fixed-day service connecting Far East and West Coast South America

TRANSIT TIMES*

EASTBOUND

SAN ANTONIO

CHANCAY

CALLAO

POSORJA

YOKOHAMA

BUSAN

SHANGHAI

NINGBO

41

36

32

29

WESTBOUND

49

45

4

LLM - PDF : 

In [24]:
# Extraction des tableaux à l'aide de la plateforme de LLM

input_file_1 = "pdfs\WAX_0.pdf"

with open(input_file_1, "r", encoding="utf-8") as f1:
    pdf_content = PdfReader('C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs\\WAX_0.pdf').pages[0]

input_file_2 = "WAX_0.md"

with open(input_file_2, "r", encoding="utf-8") as f2:
    markdown_content = f2.read()


prompt_LLM_fra = f"""Voici le contenu d'un fichier PDF pour comprendre la forme et d'un fichier Markdown. 
Extrait uniquement les tableaux de temps de transit** (Transit Times), et rends-les sous forme de 
tableaux lisibles .csv Ne commente pas, ne reformule pas.
PDF : {pdf_content}, Markdown : {markdown_content}"""

prompt_LLM = f"""This is a PDF file to understand the way it is and a Markdown file. Extract only 
Transit Times' tables and give them back to me on a readable .csv format. Don't comment and don't repharse.
PDF : {pdf_content}, Markdown : {markdown_content}"""

safe_prompt_LLM = shlex.quote(prompt_LLM)

output_file = "WAX_0.txt"

# Methode bash

#print(bash.bash(f"""http -F Bob:hiccup@ollama-sam.inria.fr/api/generate model='mistral:7b' prompt='{prompt_LLM}' > text-bis.txt"""))


# Methode subprocess

#cmd = f"""http -F Bob:hiccup@ollama-sam.inria.fr/api/generate model='mistral:7b' prompt={safe_prompt_LLM} """

#with open(output_file, "w", encoding="utf-8") as out_file:
#    subprocess.run(cmd, shell=True, stdout=out_file)

#http -F Bob:hiccup@ollama-sam.inria.fr/api/generate model="mistral:7b" prompt=prompt-LLM


# Méthode requests

url = "http://ollama-sam.inria.fr/api/generate"
data = {'model': 'mistral:7b', 'prompt': safe_prompt_LLM}
auth = ('Bob', 'hiccup')  # HTTP Basic Auth

response = requests.post(url, json=data, auth=auth)


with open(output_file, "w", encoding="utf-8") as out_file:
    out_file.write(response.text)



# Extraitre les fichiers .csv du fichier text

<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\lisef\AppData\Local\Temp\ipykernel_16024\2788557382.py:3: SyntaxWarning: invalid escape sequence '\W'
  input_file_1 = "pdfs\WAX_0.pdf"


Lecture des tableaux :

In [ ]:
# Autre script pour convertir pdf en Markdown

def pdf_tables_to_markdown(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, 1):
            tables = page.extract_tables()
            for i, table in enumerate(tables, 1):
                # Construire la table Markdown
                md = ""
                # Entête
                md += "| " + " | ".join(table[0]) + " |\n"
                md += "| " + " | ".join(["---"]*len(table[0])) + " |\n"
                # Lignes
                for row in table[1:]:
                    md += "| " + " | ".join(cell if cell else "" for cell in row) + " |\n"

                # Sauvegarder chaque tableau en markdown
                with open(f"mds_{pdf_path}", "w", encoding="utf-8") as f:
                    f.write(md)

In [ ]:
# Extraction des tableaux à l'aide de la plateforme de LLM

input_file = "test.md" #"WAX_0.md"

with open(input_file, "r", encoding="utf-8") as f:
    markdown_content = f.read()

prompt_LLM_fra = f"""Voici le contenu d'un fichier Markdown. Extrait uniquement les tableaux de temps de transit**
(Transit Times), et rends-les sous forme de tableaux lisibles .csv Ne commente pas, ne reformule pas.
Markdown : {markdown_content}"""

prompt_LLM = f"""This is a Markdown file. Extract only table and give them back to me on a readable
 .csv format. Don't comment and don't repharse. Above all else, dont add information from outside the Markdown
file ; Markdown : {markdown_content}""" #Transit Times'

safe_prompt_LLM = shlex.quote(prompt_LLM)

output_file = "test.txt" #"WAX_0.txt"




In [ ]:
input_file = "test.txt"
response_file = "test_response.txt"
output_file = "test_table.csv"

# Traitement du fichier .txt donné en réponse par le LLM
responses = []
with open(input_file, "r", encoding="utf-8") as text_file:
    for line in text_file:
        try:
            obj = json.loads(line.strip())
            if "response" in obj:
                responses.append(obj["response"])
        except json.JSONDecodeError:
            continue

full_text = "".join(responses)
full_text = full_text.replace('"""', '').replace('"', '')

with open(response_file, "w", encoding="utf-8") as file:
    file.write(full_text)


# Conversion du fichier .txt amélioré en fichier .csv
lines = [line.strip() for line in full_text.strip().splitlines() if line.strip()]

with open(output_file, "w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    for line in lines:
        writer.writerow(line.split(","))



# Vérification que le model LLM n'invente pas des données et fais simplement de l'extraction de données
# Faire un test de vérification


In [ ]:
path = 'C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs' # chemin d'accès des pdfs
subdirectories = os.listdir(path)

for fichier in subdirectories:
    if not is_html(f'C:\\Users\\lisef\\projet-buyco\\Buyco\\pdfs\\{fichier}'):
        myFile = f'pdfs\{fichier}'
        extract_last_page(myFile, f'pdfs\extract_{fichier}')
        pdf_tables_to_markdown(f"pdfs\{fichier}")
        input_file = 
        